<h1> Dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
mpl.style.use('seaborn')
from matplotlib import pyplot as plt

# Sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import classification_report

from keras import backend as K


In [2]:
# Tensorflow and Keras

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input


In [3]:
tf.data.experimental.enable_debug_mode()
tf.config.experimental_run_functions_eagerly(True)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


<h1> Import Data

In [4]:
# Load the data from the csv files

train_data = pd.read_csv("Preprocessed/preprocessed_train_3.csv", index_col=0)
test_data = pd.read_csv("Preprocessed/preprocessed_test_3.csv", index_col=0) 

In [5]:
target = "change_type"
X = train_data.drop(columns = target)
Y = train_data[target].map({'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5})

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True)

In [7]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

<h1> Basic model Keras 

In [8]:
def f1(y_true, y_pred):

    y_pred = np.argmax(y_pred.numpy(), axis = 1)

    return f1_score(y_true, y_pred, average = "macro")

def f1_loss(y_true, y_pred):

    y_true = keras.utils.to_categorical(y_true, num_classes=6)

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [15]:
input = Input(shape = np.shape(X_train)[1])
hidden1 = Dense(512, activation = "relu")(input)
hidden2 = Dense(512, activation = "relu")(hidden1)
hidden3 = Dense(64, activation = "relu")(hidden2)
hidden4 = Dense(64, activation = "elu")(hidden3)
output = Dense(6, activation = "softmax")(hidden4)

model = keras.models.Model(inputs = [input], outputs = [output])

model.compile(optimizer="adam",  metrics=["accuracy"], loss = keras.losses.SparseCategoricalCrossentropy(), run_eagerly=True)

In [16]:
history = model.fit(X_train, y_train.to_numpy(), epochs = 10, batch_size = 256, validation_split=0.2, class_weight="balanced")

AttributeError: 'str' object has no attribute 'keys'

In [ ]:
y_pred = model.predict(X_test)

print(classification_report(np.argmax(y_pred, axis =1), y_test))
cf_matrix = confusion_matrix(np.argmax(y_pred, axis =1), y_test)
print(cf_matrix)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.53      0.69     77502
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

    accuracy                           0.53     77502
   macro avg       0.17      0.09      0.12     77502
weighted avg       1.00      0.53      0.69     77502

[[    0     0     0     0     0     0]
 [    0     0     0     0     0     0]
 [ 7581  3679 41024 24779   405    34]
 [    0     0     0     0     0     0]
 [    0     0     0     0     0     0]
 [    0     0     0     0     0     0]]


C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r